In [887]:
import numpy as np
from docopt import docopt
from scipy import sparse
import os
import subprocess
from shutil import copyfile
from operator import itemgetter

In [888]:
'''
workflow of the verify_implementation script

verify glove bias
1. I build W and H matrix, build bias term. And then, multply W and H, add bias term, use this matrix as my question
2. store the question matrix nnz term and meta information to ./matrix_folder
3. build a weight matrix and store its question and nnz term to ./count_folder
4. modified code, let it output bias term when factorize the matrix
5. read in the answer matrix, test them, to see whether it can output the bias term


verify glove weight
1. I build W and H matrix, multply W and H, use this matrix as my question
2. build a weight matrix, assign one position very high weight, store nnz and meta file in count_folder
3. open glove weight, close glove bias, factorize the matrix
4. 5. read in the answer matrix, test them, to see the error on the position with high weight






'''

'\nworkflow of the verify_implementation script\n\nverify glove bias\n1. I build W and H matrix, build bias term. And then, multply W and H, add bias term, use this matrix as my question\n2. store the question matrix nnz term and meta information to ./matrix_folder\n3. build a weight matrix and store its question and nnz term to ./count_folder\n4. modified code, let it output bias term when factorize the matrix\n5. read in the answer matrix, test them, to see whether it can output the bias term\n\n\nverify glove weight\n\n\n\n\n\n\n'

In [889]:
def build_answer_and_question(matrix_size, embedding_rank, include_bias = 0):
    W = np.random.random((matrix_size, embedding_rank))
    H = np.random.random((embedding_rank, matrix_size))

#     W = W*(W>0.55)
#     H = H*(H<0.45)
    
    W_bias = np.random.random(matrix_size)
    H_bias = np.random.random(matrix_size)
    
    question_matrix = np.dot(W, H)
    
    if include_bias:

        print "inlcude_bias is on"
        question_matrix = np.transpose(question_matrix)
        for i in range(question_matrix.shape[0]):
            question_matrix[i] = np.add(question_matrix[i], W_bias)
        question_matrix = np.transpose(question_matrix)
        
        for i in range(question_matrix.shape[1]):
            question_matrix[i] = np.add(question_matrix[i], H_bias)
    
    
    
    print question_matrix[1][5]
    print (np.dot(W[1], H[:,5]) + W_bias[1] + H_bias[5])
    return W, H ,W_bias, H_bias, question_matrix

In [894]:
def build_weight_matrix(matrix_size, weight_for_exception_term):
    weight_matrix = np.ones((matrix_size, matrix_size))
    weight_matrix[1][1] = 1
    
    return weight_matrix

In [891]:
def save_nonzero_term_fast(nparray, path_and_name):
    csrmatrix = sparse.csr_matrix(nparray)  
    with open(path_and_name,'w') as f:
        for i in range(len(csrmatrix.indptr)-1):
#             if i % 1000000 == 0:
#                 print i, len(csrmatrix.indptr)-1
            columnIndices=[]
            dataInLine=[]
            columnIndices=csrmatrix.indices[csrmatrix.indptr[i]:csrmatrix.indptr[i+1]] 
            dataInLine=csrmatrix.data[csrmatrix.indptr[i]:csrmatrix.indptr[i+1]]
            for j in range(len(columnIndices)):
                f.write("%d %d %.6f\n"% (i+1, columnIndices[j]+1, dataInLine[j]))         

In [892]:
# 71290 71290
# 0 training.ratings
# 0 test.ratings
def create_meta_file(nparray, path_and_name):
    csrmatrix = sparse.csr_matrix(nparray)
    nnz_number = csrmatrix.getnnz()
    size = nparray.shape[0]

    with open(path_and_name,'w') as f:
        f.write("%d %d\n" %(size, size))
        f.write("%d %s\n" %(nnz_number, 'training.ratings'))
        f.write("%d %s" %(nnz_number, 'test.ratings'))

In [874]:
def read_answer(path_and_name):
    a = np.loadtxt(path_and_name +'.W')
#     print ("a.shape",a.shape)
    b = np.transpose(np.loadtxt(path_and_name +'.H'))
#     print ("b.shape",b.shape)
    return a, b

In [897]:
#verify glove weight
matrix_size = 8
rank = 6

W, H ,W_bias, H_bias, question_matrix = build_answer_and_question(matrix_size, rank, 0)
create_meta_file(question_matrix, "./matrix_folder/meta")
save_nonzero_term_fast(question_matrix, "./matrix_folder/training.ratings")
save_nonzero_term_fast(question_matrix, "./matrix_folder/test.ratings")

weight_matrix = build_weight_matrix(matrix_size, 1000)
create_meta_file(weight_matrix, "./count_folder/meta")
save_nonzero_term_fast(weight_matrix, "./count_folder/training.ratings")
save_nonzero_term_fast(weight_matrix, "./count_folder/test.ratings")

subprocess.check_output(["make"])
subprocess.check_output(["./converter", "./matrix_folder"])
subprocess.check_output(["./omp-pmf-train", "-s", "10", "-n", "10", "-f", "1", "-t", "20", "-q", "1", "-p", "0", "-r", "0.015625", "-l", "0.015625", "-b", "0", "-k", str(rank), "-E", "0", "-X", "1000000", "-W", "1", "-G", "0", "matrix_folder", "count_folder", "test_code"])

W_answer, H_answer = read_answer('test_code-l0.015625-r0.015625-iter20-gweight1-xmax1000000-gbias0.final')
subprocess.check_output(["rm", 'test_code-l0.015625-r0.015625-iter20-gweight1-xmax1000000-gbias0.final.W'])
subprocess.check_output(["rm", 'test_code-l0.015625-r0.015625-iter20-gweight1-xmax1000000-gbias0.final.H'])

print np.subtract(question_matrix, np.dot(W_answer, H_answer))

1.66432280421
2.23205736607
[[ 1.948  1.207  1.022  1.086  0.74   1.841  1.261  0.853]
 [ 1.557  0.899  1.423  0.82   1.182  1.664  1.272  1.037]
 [ 2.226  1.544  1.73   1.185  1.545  2.668  1.981  1.356]
 [ 2.017  1.199  1.564  1.22   1.441  1.681  1.142  1.166]
 [ 1.03   0.696  0.654  0.5    0.775  1.051  0.767  0.513]
 [ 1.536  1.204  1.27   0.977  1.647  1.802  1.268  0.987]
 [ 2.187  1.495  1.485  1.431  1.506  2.159  1.436  1.185]
 [ 1.707  1.298  1.146  0.977  1.223  1.761  1.205  0.939]]


In [885]:
#verify glove bias

# def main():
#     args = docopt("""
#     Usage: 
#         verify_inner_product.py <representation_shared_path>
#     """)

matrix_size = 8
rank = 5
W, H ,W_bias, H_bias, question_matrix = build_answer_and_question(matrix_size, rank, 1)

# print question_matrix

create_meta_file(question_matrix, "./matrix_folder/meta")
save_nonzero_term_fast(question_matrix, "./matrix_folder/training.ratings")
save_nonzero_term_fast(question_matrix, "./matrix_folder/test.ratings")

create_meta_file(question_matrix, "./count_folder/meta")
save_nonzero_term_fast(question_matrix, "./count_folder/training.ratings")
save_nonzero_term_fast(question_matrix, "./matrix_folder/test.ratings")

subprocess.check_output(["make"])
subprocess.check_output(["./converter", "./matrix_folder"])
subprocess.check_output(["./omp-pmf-train", "-s", "10", "-n", "10", "-f", "1", "-t", "10", "-q", "1", "-p", "0", "-r", "0.015625", "-l", "0.03125", "-b", "0", "-k", str(rank), "-E", "0", "-X", "1", "-W", "0", "-G", "2", "matrix_folder", "count_folder", "test_code"])

W_answer, H_answer = read_answer('test_code-l0.031250-r0.015625-iter10-gweight0-xmax1-gbias2.final')
subprocess.check_output(["rm", 'test_code-l0.031250-r0.015625-iter10-gweight0-xmax1-gbias2.final.W'])
subprocess.check_output(["rm", 'test_code-l0.031250-r0.015625-iter10-gweight0-xmax1-gbias2.final.H'])

print W_bias
print W_answer[:,rank+1]

print H_bias
print H_answer[rank,:]

W_bias_dict = dict([(i, a) for i, a in enumerate(W_bias)])
sort_W_bias_dict = sorted(W_bias_dict.items(), key=itemgetter(1))
print sort_W_bias_dict

W_answer_dict = dict([(i, a) for i, a in enumerate(W_answer[:,rank+1])])
sort_W_answer_dict = sorted(W_answer_dict.items(), key=itemgetter(1))
print sort_W_answer_dict

H_bias_dict = dict([(i, a) for i, a in enumerate(H_bias)])
sort_H_bias_dict = sorted(H_bias_dict.items(), key=itemgetter(1))
# print H_bias_dict

H_answer_dict = dict([(i, a) for i, a in enumerate(H_answer[rank,:])])
sort_H_answer_dict = sorted(H_answer_dict.items(), key=itemgetter(1))
# print H_answer_dict

# print np.subtract(W_bias, W_answer[:,rank+1])
# print np.subtract(H_bias, H_answer[rank,:])

inlcude_bias is on
2.11814568245
2.11814568245
[ 0.742  0.035  0.565  0.788  0.807  0.797  0.974  0.709]
[ 0.025 -0.085  0.064  0.018  0.001  0.001  0.083  0.05 ]
[ 0.884  0.645  0.792  0.581  0.905  0.975  0.075  0.2  ]
[ 0.17   0.148  0.203  0.146  0.222  0.197  0.06   0.105]
[(1, 0.034687117733612549), (2, 0.5646343551039168), (7, 0.70892671481778602), (0, 0.74154172135293617), (3, 0.78792273763768861), (5, 0.79665764254060456), (4, 0.80666487565685507), (6, 0.9739816139185018)]
[(1, -0.084616999999999998), (5, 0.001132), (4, 0.0014790000000000001), (3, 0.017877000000000001), (0, 0.025127), (7, 0.049993000000000003), (2, 0.064140000000000003), (6, 0.082632999999999998)]
